<a href="https://colab.research.google.com/github/aymankhchman/Litllte_Project/blob/master/ideas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [3]:
import torch

def perplexity(text):
  inputs = tokenizer(text, return_tensors='pt')
  with torch.no_grad():
    outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss
  return torch.exp(loss).item(), outputs

In [4]:
transcript = "I need to buy something at the store."


In [5]:
p, o = perplexity(transcript)

In [12]:
dir(o)

['__annotations__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__post_init__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__ror__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'attentions',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'hidden_states',
 'items',
 'keys',
 'logits',
 'loss',
 'move_to_end',
 'past_key_values',
 'pop',
 'popitem',
 'setdefault',
 'to_tuple',
 'update',
 'values']

In [16]:
o.logits

tensor([[[ 0.9805,  1.8203, -0.3262,  ..., -7.6250, -7.6250, -7.6250],
         [12.1250, 11.5625, 11.4375,  ...,  5.9688,  5.9688,  5.9688],
         [ 1.5312,  1.2578, -0.6016,  ..., -5.7812, -5.7812, -5.7812],
         ...,
         [ 0.6836,  2.7969, -2.3125,  ..., -6.6250, -6.6250, -6.6250],
         [10.4375,  6.8438,  0.4102,  ..., -2.3750, -2.3750, -2.3750],
         [-3.2031, -3.2031, -3.2500,  ..., -6.1250, -6.1250, -6.1250]]],
       dtype=torch.bfloat16)

In [18]:
import torch.nn.functional as F

In [22]:
probs = F.softmax(o.logits, dim=-1)

In [30]:
import torch


probs[0, torch.arange(probs.shape[1]), inputs["input_ids"]]

tensor([[4.9770e-06, 7.4804e-06, 2.7466e-04, 6.0797e-06, 7.6294e-06, 4.6730e-05,
         1.8835e-05, 4.0047e-07, 2.2555e-10]], dtype=torch.bfloat16)

In [45]:
for i in range(len(inputs["input_ids"][0])):
  id = inputs["input_ids"][0][i].item()
  print(tokenizer.convert_ids_to_tokens(id))

i
Ġneed
Ġto
Ġbuy
e
Ġsum
thing
Ġat
Ġthe
Ġst
oure


In [34]:
inputs["input"]

tensor([  40, 1184,  311, 3695, 2494,  518,  279, 3553,   13])

In [6]:
o.logits.shape

torch.Size([1, 9, 152064])

In [41]:
inputs = tokenizer('i need to buye sumthing at the stoure', return_tensors="pt", return_offsets_mapping=True)

In [42]:
inputs["input_ids"][0][1]

tensor(1184)

In [44]:
inputs["offset_mapping"]

tensor([[[ 0,  1],
         [ 1,  6],
         [ 6,  9],
         [ 9, 13],
         [13, 14],
         [14, 18],
         [18, 23],
         [23, 26],
         [26, 30],
         [30, 33],
         [33, 37]]])

In [47]:
tokenizer.byte_encoder

AttributeError: Qwen2TokenizerFast has no attribute byte_encoder

In [66]:
tokens = tokenizer.tokenize('i need to buye sumthing at the stoure.', return_tensors="pt", return_offsets_mapping=True)

In [68]:
inputs = tokenizer('i need to buye sumthing at the stoure.', return_tensors="pt", return_offsets_mapping=True, add_special_tokens=False)

In [69]:
inputs["input_ids"].shape

torch.Size([1, 12])

In [65]:
len(tokens)

12

In [71]:
input_ids = inputs["input_ids"]

with torch.no_grad():
    outputs = model(input_ids=input_ids, labels=input_ids)
logits = outputs.logits  # Shape: (1, seq_len, vocab_size)

In [73]:
logits.shape

torch.Size([1, 12, 152064])

In [74]:
# Shift logits and labels for alignment
shift_logits = logits[..., :-1, :].contiguous()  # Predictions for tokens 1...n
shift_labels = input_ids[..., 1:].contiguous()   # Ground truth for tokens 1...n

In [75]:
log_probs = torch.nn.functional.log_softmax(shift_logits, dim=-1)


In [78]:
log_probs_for_labels = torch.gather(log_probs, -1, shift_labels.unsqueeze(-1)).squeeze(-1)
probs = torch.exp(log_probs_for_labels).squeeze(0).tolist()

In [81]:
probs

[0.02587890625,
 0.6328125,
 0.00075531005859375,
 8.307397365570068e-07,
 1.5735626220703125e-05,
 0.11376953125,
 0.01300048828125,
 0.380859375,
 0.004791259765625,
 0.00150299072265625,
 0.09033203125]

In [83]:
import numpy as np

def group_tokens_into_words(tokens, probs):
    word_probs = []
    current_word = []
    current_word_log_prob = 0

    for token, prob in zip(tokens[1:], probs):  # Skip the first token (no context)
        if token.startswith("Ġ"):
            # Save previous word
            if current_word:
                word_text = "".join(current_word)
                word_prob = np.exp(current_word_log_prob)
                word_probs.append((word_text, word_prob))
            # Start new word
            current_word = [token[1:]]  # Remove "Ġ"
            current_word_log_prob = np.log(prob)
        else:
            # Append subword (e.g., "thing" to "Ġsum" → "something")
            current_word.append(token)
            current_word_log_prob += np.log(prob)

    # Add the last word
    if current_word:
        word_text = "".join(current_word)
        word_prob = np.exp(current_word_log_prob)
        word_probs.append((word_text, word_prob))

    return word_probs

word_probs = group_tokens_into_words(tokens, probs)
print("Word probabilities:", word_probs)

Word probabilities: [('need', np.float64(0.02587890625)), ('to', np.float64(0.6328125)), ('buye', np.float64(6.274660790950296e-10)), ('sumthing', np.float64(1.7902348190546019e-06)), ('at', np.float64(0.01300048828125)), ('the', np.float64(0.380859375)), ('stoure.', np.float64(6.505007377199939e-07))]


In [84]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
import torch

model_name = "roberta-large"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForMaskedLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [101]:
text = "I need to bye sumthing at the store."
input_ids = tokenizer.encode(text, return_tensors="pt")[0]
tokens = tokenizer.convert_ids_to_tokens(input_ids)

print("Tokens:", tokens)
# Example output: ['<s>', 'I', 'Ġneed', 'Ġto', 'Ġbye', 'Ġsum', 'thing', 'Ġat', 'Ġthe', 'Ġstore', '.', '</s>']

Tokens: ['<s>', 'I', 'Ġneed', 'Ġto', 'Ġbye', 'Ġsum', 'thing', 'Ġat', 'Ġthe', 'Ġstore', '.', '</s>']


In [92]:
tokens

['I',
 'Ġneed',
 'Ġto',
 'Ġbye',
 'Ġ[',
 'MAS',
 'K',
 ']',
 'Ġat',
 'Ġthe',
 'Ġstore',
 '.']

In [93]:
with torch.no_grad():
    outputs = model(**inputs)
logits = outputs.logits  # Shape: (1, seq_len, vocab_size)

# Compute probabilities for all tokens
probs = torch.softmax(logits, dim=-1).squeeze(0)  # Shape: (seq_len, vocab_size)
token_probs = probs[torch.arange(len(input_ids)), input_ids].tolist()  # Probability of each token in context

In [94]:
token_probs

[7.759028449072503e-06,
 0.0023050664458423853,
 0.9995896220207214,
 0.9993770718574524,
 0.9997859597206116,
 0.999990701675415,
 0.9999933242797852,
 0.9999064207077026,
 0.999932050704956,
 0.9999147653579712,
 0.9998819828033447,
 0.8763456344604492]

In [102]:
def compute_token_probabilities(text, tokenizer, model):
    input_ids = tokenizer.encode(text, return_tensors="pt")[0]
    seq_len = len(input_ids)
    token_probs = []

    for i in range(seq_len):
        # Mask the i-th token
        masked_input_ids = input_ids.clone()
        masked_input_ids[i] = tokenizer.mask_token_id

        with torch.no_grad():
            outputs = model(masked_input_ids.unsqueeze(0))

        # Get probabilities for the original token
        probs = torch.softmax(outputs.logits[0, i], dim=-1)
        original_token_id = input_ids[i].item()
        token_prob = probs[original_token_id].item()
        token_probs.append(token_prob)

    return token_probs

token_probs = compute_token_probabilities(text, tokenizer, model)
print("Token probabilities:", token_probs)

Token probabilities: [0.9999898672103882, 0.34555643796920776, 0.028702370822429657, 0.944093644618988, 1.1479529575808556e-06, 0.013333488255739212, 0.002710553351789713, 0.3426727056503296, 0.9314698576927185, 0.0025335033424198627, 0.7727766633033752, 0.9999096393585205]


In [104]:
def group_tokens_into_words(tokens, token_probs):
    word_probs = []
    current_word = []
    current_word_log_prob = 0

    for token, prob in zip(tokens, token_probs):
        if token.startswith("Ġ"):
            # Save previous word
            if current_word:
                word_prob = np.exp(current_word_log_prob)
                word_text = "".join(current_word)
                word_probs.append((word_text, word_prob))
            # Start new word
            current_word = [token[1:]]  # Remove "Ġ"
            current_word_log_prob = np.log(prob)
        else:
            # Append subword (e.g., "thing" to "sum")
            current_word.append(token)
            current_word_log_prob += np.log(prob)

    # Add the last word
    if current_word:
        word_prob = np.exp(current_word_log_prob)
        word_text = "".join(current_word)
        word_probs.append((word_text, word_prob))

    return word_probs

word_probs = group_tokens_into_words(tokens, token_probs)
low_prob_words = [word for word in word_probs if word[1] < 0.1]
print("Low probability words:", low_prob_words)

Low probability words: [('need', np.float64(0.02870237082242965)), ('bye', np.float64(1.147952957580855e-06)), ('sumthing', np.float64(3.614113128264274e-05)), ('store.</s>', np.float64(0.0019576553484442994))]


In [105]:
word_probs

[('<s>I', np.float64(0.3455529365185228)),
 ('need', np.float64(0.02870237082242965)),
 ('to', np.float64(0.944093644618988)),
 ('bye', np.float64(1.147952957580855e-06)),
 ('sumthing', np.float64(3.614113128264274e-05)),
 ('at', np.float64(0.3426727056503296)),
 ('the', np.float64(0.9314698576927185)),
 ('store.</s>', np.float64(0.0019576553484442994))]

In [100]:
list(zip(tokens, token_probs))

[('I', 0.0009080779273062944),
 ('Ġneed', 0.0005939671536907554),
 ('Ġto', 0.0015326812863349915),
 ('Ġbye', 1.1398414017094183e-06),
 ('Ġsum', 0.004855589009821415),
 ('thing', 0.0003946122305933386),
 ('Ġat', 0.16179360449314117),
 ('Ġthe', 0.0825163722038269),
 ('Ġstore', 0.014337860979139805),
 ('.', 6.985579148022225e-06)]

In [125]:
from transformers import RobertaTokenizer, RobertaForMaskedLM, AutoModelForMaskedLM, AutoTokenizer
import torch
import numpy as np

# Load RoBERTa
model_name = "roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

text = "I need to bye sumthing at the store."

# Compute token probabilities (with special tokens handled)
def compute_token_probabilities(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt")
    input_ids = inputs["input_ids"][0]
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Remove special tokens
    filtered_input_ids = input_ids[1:-1]
    filtered_tokens = tokens[1:-1]
    seq_len = len(filtered_input_ids)

    token_probs = []
    for i in range(seq_len):
        masked_input_ids = filtered_input_ids.clone()
        masked_input_ids[i] = tokenizer.mask_token_id

        # Add back special tokens for proper processing
        masked_input_ids = torch.cat([
            torch.tensor([tokenizer.bos_token_id]),
            masked_input_ids,
            torch.tensor([tokenizer.eos_token_id])
        ]).unsqueeze(0)

        with torch.no_grad():
            outputs = model(masked_input_ids)

        # Extract probability for the original token
        probs = torch.softmax(outputs.logits[0, i+1], dim=-1)
        token_prob = probs[filtered_input_ids[i]].item()
        token_probs.append(token_prob)

    return filtered_tokens, token_probs

# Group tokens into words
def group_tokens_into_words(tokens, token_probs):
    word_probs = []
    current_word = []
    current_word_log_prob = 0

    for token, prob in zip(tokens, token_probs):
        if token.startswith("Ġ"):
            if current_word:
                word_prob = np.exp(current_word_log_prob)
                word_text = "".join(current_word)
                word_probs.append((word_text, word_prob))
            current_word = [token[1:]]
            current_word_log_prob = np.log(prob)
        else:
            current_word.append(token)
            current_word_log_prob += np.log(prob)

    if current_word:
        word_prob = np.exp(current_word_log_prob)
        word_text = "".join(current_word)
        word_probs.append((word_text, word_prob))

    return word_probs

# Flag low-probability words

text = "Aye am going to bye sum bread at the supermarket"
tokens, token_probs = compute_token_probabilities(text, tokenizer, model)
word_probs = group_tokens_into_words(tokens, token_probs)
low_prob_words = [word for word in word_probs if word[1] < 1e-4]
print("Low probability words:", low_prob_words)

Low probability words: [('bye', np.float64(2.8672227927017954e-05)), ('sum', np.float64(2.5788645871216445e-05))]


In [123]:
word_probs

[('▁Aye▁am▁going▁to▁bye▁sum▁bread▁at▁the▁supermarket',
  np.float64(3.7054239237798192e-25))]

In [124]:
  tokens

['▁A',
 'ye',
 '▁am',
 '▁going',
 '▁to',
 '▁by',
 'e',
 '▁sum',
 '▁bread',
 '▁at',
 '▁the',
 '▁supermarket']

In [126]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

model_name = "FacebookAI/xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)


text = "Aye am going to bye sum bread at the supermarket"
tokens = tokenizer.tokenize(text)
print("Tokens:", tokens)

# Output: ['▁A', 'ye', '▁am', '▁going', '▁to', '▁by', 'e', '▁sum', '▁bread', '▁at', '▁the', '▁supermarket']

def group_tokens_into_words(tokens, token_probs):
    word_probs = []
    current_word = []
    current_word_log_prob = 0

    for token, prob in zip(tokens, token_probs):
        if token.startswith("▁"):
            # Save previous word
            if current_word:
                word_prob = np.exp(current_word_log_prob)
                word_text = "".join(current_word)
                word_probs.append((word_text, word_prob))
            # Start new word
            current_word = [token[1:]]  # Remove "▁"
            current_word_log_prob = np.log(prob)
        else:
            # Append subword (e.g., "ye" to "A" → "Aye")
            current_word.append(token)
            current_word_log_prob += np.log(prob)

    # Add the last word
    if current_word:
        word_prob = np.exp(current_word_log_prob)
        word_text = "".join(current_word)
        word_probs.append((word_text, word_prob))

    return word_probs

def compute_token_probabilities(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt")
    input_ids = inputs["input_ids"][0]
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Remove special tokens ([CLS] and [SEP])
    filtered_input_ids = input_ids[1:-1]
    filtered_tokens = tokens[1:-1]
    seq_len = len(filtered_input_ids)

    token_probs = []
    for i in range(seq_len):
        masked_input_ids = filtered_input_ids.clone()
        masked_input_ids[i] = tokenizer.mask_token_id

        # Add back special tokens for proper processing
        masked_input_ids = torch.cat([
            torch.tensor([tokenizer.cls_token_id]),
            masked_input_ids,
            torch.tensor([tokenizer.sep_token_id])
        ]).unsqueeze(0)

        with torch.no_grad():
            outputs = model(masked_input_ids)

        # Extract probability for the original token
        probs = torch.softmax(outputs.logits[0, i+1], dim=-1)
        token_prob = probs[filtered_input_ids[i]].item()
        token_probs.append(token_prob)

    return filtered_tokens, token_probs

text = "Aye am going to bye sum bread at the supermarket"
tokens, token_probs = compute_token_probabilities(text, tokenizer, model)
word_probs = group_tokens_into_words(tokens, token_probs)
low_prob_words = [word for word in word_probs if word[1] < 1e-4]
print("Low probability words:", low_prob_words)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the model checkpoint at FacebookAI/xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Tokens: ['▁A', 'ye', '▁am', '▁going', '▁to', '▁by', 'e', '▁sum', '▁bread', '▁at', '▁the', '▁supermarket']
Low probability words: [('Aye', np.float64(4.884878340752589e-05)), ('bye', np.float64(3.7477039864430496e-05)), ('sum', np.float64(8.329535376105918e-06))]


In [127]:
word_probs

[('Aye', np.float64(4.884878340752589e-05)),
 ('am', np.float64(0.038117233663797385)),
 ('going', np.float64(0.5207866430282593)),
 ('to', np.float64(0.8063057065010071)),
 ('bye', np.float64(3.7477039864430496e-05)),
 ('sum', np.float64(8.329535376105918e-06)),
 ('bread', np.float64(0.0003529246896505355)),
 ('at', np.float64(0.41035524010658264)),
 ('the', np.float64(0.7144502401351929)),
 ('supermarket', np.float64(0.007961824536323547))]